# Basics about quantum simulation

In [8]:
using Yao, Yao.Blocks
using LinearAlgebra
using Test, BenchmarkTools

## Basic Register Operations

### Constructors

In [9]:
ψ1 = zero_state(5)
ψ2 = zero_state(5, 3)  # register with batch.

DefaultRegister{3, Complex{Float64}}
    active qubits: 5/5

In [10]:
@assert repeat(ψ1, 3) == ψ2
repeat(ψ2, 3)   # repeat registers in batch dimension

DefaultRegister{9, Complex{Float64}}
    active qubits: 5/5

In [11]:
join(ψ2, ψ2)

DefaultRegister{3, Complex{Float64}}
    active qubits: 10/10

In [12]:
ψ1 = rand_state(5)
ψ2 = rand_state(5)
@assert ψ1 |> isnormalized && ψ2 |> isnormalized
ψ3 = 0.3ψ1 + 2ψ2
@assert ψ3 |> isnormalized == false
@assert ψ3 |> normalize! |> isnormalized

In [13]:
reg = product_state(5, 0b11100)

DefaultRegister{1, Complex{Float64}}
    active qubits: 5/5

In [14]:
addbit!(copy(reg), 2) == product_state(7, 0b0011100)

true

### Adjoint

In [15]:
bra = rand_state(3)'

DefaultRegister{1, Complex{Float64}}
    active qubits: 3/3 [†]

In [16]:
@assert reg |> isnormalized
reg' * reg

1.0 + 0.0im

# broadcasting & vectorization

In [17]:
map(reg->reg |> put(5, 2=>X), repeat(ψ2, 3))

3-element Array{DefaultRegister{1,Complex{Float64},SubArray{Complex{Float64},2,Array{Complex{Float64},3},Tuple{Base.Slice{Base.OneTo{Int64}},Base.Slice{Base.OneTo{Int64}},Int64},true}},1}:
 DefaultRegister{1, Complex{Float64}}
    active qubits: 5/5
 DefaultRegister{1, Complex{Float64}}
    active qubits: 5/5
 DefaultRegister{1, Complex{Float64}}
    active qubits: 5/5

# Focus
To support modulized design of circuits, we use `focus!` to set some qubits as active, and the remaining are not supported!

In [35]:
focus!(reg, 1:5)

DimensionMismatch: DimensionMismatch("array size 48 must be divisible by the product of the new dimensions (32, Colon())")

In [36]:
nactive(reg)

0

In [64]:
relax!(reg)

DefaultRegister{1, Complex{Float64}}
    active qubits: 5/5

In [65]:
focus!(reg, [3,1,5]) do r
    r.state = randn(8, 8) * r.state
    r
end

DefaultRegister{1, Complex{Float64}}
    active qubits: 5/5

# Measure

In [18]:
reg = rand_state(7, 3)
measure(reg, 3)

3×3 Array{Int64,2}:
  22   30  122
 108  125   74
 119   34   74

In [19]:
[measure!(reg) for i=1:3]

3-element Array{Array{Int64,1},1}:
 [26, 65, 105]
 [26, 65, 105]
 [26, 65, 105]

In [20]:
[measure_reset!(reg, val=0b0111000) for i=1:3]

3-element Array{Array{Int64,1},1}:
 [26, 65, 105]
 [56, 56, 56] 
 [56, 56, 56] 

In [21]:
[measure_reset!(focus!(reg, 1:3), val=0b011) for i=1:3]

3-element Array{Array{Int64,1},1}:
 [0, 0, 0]
 [3, 3, 3]
 [3, 3, 3]

In [22]:
reg

DefaultRegister{3, Complex{Float64}}
    active qubits: 3/7

In [23]:
reg = rand_state(5)
reg3 = relax!(reg) ⊗ rand_state(4)

DefaultRegister{1, Complex{Float64}}
    active qubits: 9/9

In [24]:
select(reg3, 0b011100010) |> statevec

1-element Array{Complex{Float64},1}:
 -0.0007265665976154684 - 0.002373871964352342im

In [25]:
select(repeat(reg3, 3) |> focus!(1,4,2), 0b110) |> statevec

1×192 Array{Complex{Float64},2}:
 0.00347749+0.00134799im  -0.000974028+0.0050492im  …  0.00644002-0.020048im

In [26]:
reg1 = rand_state(3,3)
reg2 = rand_state(3,3)
join(reg1, reg2) 

DefaultRegister{3, Complex{Float64}}
    active qubits: 6/6

In [31]:
reg = rand_state(3)
addbit!(copy(reg), 3) ≈ join(reg, zero_state(3))

true

# Utilities

In [32]:
ψ1 = rand_state(6, 3)
ψ2 = rand_state(6, 3)
fidelity(ψ1, ψ2)

3-element Array{Float64,1}:
 0.13911050484863208
 0.06541252573815   
 0.1793275044952917 

In [33]:
focus!(ψ1, [1,2,4])' * focus!(ψ2, [1,2,4])

3-element Array{Array{Complex{Float64},2},1}:
 [-0.00511302-0.00872987im -0.0370747-0.00201094im … 0.0269886+0.0664759im -0.0220408+0.0325978im; -0.034716-0.0129275im 0.0712498-0.0176278im … 0.00445961-0.00805039im 0.0305373+0.0291869im; … ; -0.0226332-0.0259967im -0.00803662+0.00555399im … 0.0112411-0.00355682im -0.0346827-0.00579148im; 0.0114274-0.0214105im -0.00583291+0.000909081im … -0.00492019+0.0145968im -0.00211849-0.00168544im]
 [0.0514252+0.0275375im -0.0289289+0.00234231im … -0.033624-0.00907344im 0.0430125+0.0368256im; -0.0147504-0.0605127im -0.0504855-0.00561207im … 0.0414246+0.049943im 0.044266-0.100863im; … ; 0.018502-0.0697267im 0.0407958+0.0670524im … -0.0490417+0.0568143im 0.0209275+0.0202304im; -0.0167236+0.0209528im -0.0174196-0.0112465im … 0.00926956+0.0238217im 0.0134241-0.023464im]                   
 [0.0229517+0.0163829im -0.00629934-0.0339095im … 0.0562731-0.0526053im 0.0468288+0.0423874im; 0.00234255+0.0192942im 0.0280416-0.0265151im … 0.0483064+0.0121646im 0.

# Batched Registers

In [34]:
reg = rand_state(10, 8)

DefaultRegister{8, Complex{Float64}}
    active qubits: 10/10

In [35]:
@. reg * 5 - 4 * reg ≈ reg

8-element BitArray{1}:
 true
 true
 true
 true
 true
 true
 true
 true

# Blocks

In [36]:
X

X gate

In [37]:
mat(X)

2×2 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       1.0+0.0im
 1.0+0.0im     0     

In [38]:
px = put(3, 1=>X)

Total: 3, DataType: Complex{Float64}
put on (1)
└─ X gate


In [39]:
mat(px)

8×8 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       1.0+0.0im     0       …     0          0          0     
 1.0+0.0im     0          0             0          0          0     
    0          0          0             0          0          0     
    0          0       1.0+0.0im        0          0          0     
    0          0          0          1.0+0.0im     0          0     
    0          0          0       …     0          0          0     
    0          0          0             0          0       1.0+0.0im
    0          0          0             0       1.0+0.0im     0     

In [40]:
2X

Total: 1, DataType: Complex{Float64}
[2] X gate


In [41]:
mat(2X)

2×2 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
    0       2.0+0.0im
 2.0+0.0im     0     

In [42]:
cx = control(3, 3, 1=>X)

Total: 3, DataType: Complex{Float64}
control(3)
└─ (1,)=>X gate


In [43]:
mat(cx)

8×8 LuxurySparse.PermMatrix{Complex{Float64},Int64,Array{Complex{Float64},1},Array{Int64,1}}:
 1.0+0.0im     0          0       …     0          0          0     
    0       1.0+0.0im     0             0          0          0     
    0          0       1.0+0.0im        0          0          0     
    0          0          0             0          0          0     
    0          0          0          1.0+0.0im     0          0     
    0          0          0       …     0          0          0     
    0          0          0             0          0       1.0+0.0im
    0          0          0             0       1.0+0.0im     0     